In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load in dependencies
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#read in data
path = "/content/drive/Shareddrives/CVFinalProject/weights/"

def get_loss_list(folder_path, epoch1=50, epoch2=100):
    first_run_train = np.load(folder_path+"/training_losses.npy")

    try:
      first_run_test = np.load(folder_path+"/testing_losses.npy")
      second_run_train = np.load(folder_path+"/training_losses1.npy")
      second_run_test = np.load(folder_path+"/testing_losses1.npy")
    except:
      pass

    print(folder_path)
    try:
      print(f"train_1:{first_run_train.shape}")
      print(f"test_1:{first_run_test.shape}")
      print(f"train_2:{second_run_train.shape}")
      print(f"test_2:{second_run_test.shape}")
    except:
      pass

    def reduce_list(dataset, epochNum):
      tempList = []
      epochLoss = []
      batchSize = len(dataset)/epochNum

      for i in range(len(dataset)):
        tempList.append(dataset[i])
        if (i+1) % batchSize == 0:
          epochLoss.append(np.array(tempList).mean())
          tempList = []

      return epochLoss
    
    #train loss
    train_loss = reduce_list(first_run_train, epoch1)
    try:
      #test loss
       test_loss = reduce_list(first_run_test, epoch1)
    except:
      test_loss = []

    try:
       #second train loss
       strain_loss = reduce_list(second_run_train,epoch2)
       train_loss.extend(strain_loss)

       #second test loss
       stest_loss = reduce_list(second_run_test, epoch2)
       test_loss.extend(strain_loss)
    except:
      pass

    return np.array(train_loss), np.array(test_loss)

In [ ]:
train_loss = []
test_loss = []
#get the train and test loss
for i in range(1,8):
  if i == 1 or i == 2:
    epoch_num = 50
  else:
    epoch_num = 100
  result = get_loss_list(f"{path}experiment{i}", epoch1=epoch_num)
  train_loss.append(result[0])
  test_loss.append(result[1])

In [ ]:
def plot_loss(train_loss, test_loss, title):
  if len(test_loss) != 0:
    assert len(train_loss) == len(test_loss)
    ylabel = "Train/test Loss" 
  else:
    ylabel = "Train Loss"

  epoch_num = np.arange(len(train_loss))
  plt.plot(epoch_num, train_loss, label="train")
  try:
    plt.plot(epoch_num, test_loss, label="test")
  except:
    pass

  plt.xlabel('Num of Epoch')
  plt.ylabel(ylabel)
  plt.title(title)
  plt.legend()
  plt.show()

In [ ]:
ex_titles = ["Koalarization on dataset 1",
             "U-net on dataset 1",
             "CGAN with pretrained U-net weights on dataset 1",
             "Koalarization on dataset 2",
             "U-net on dataset 2",
             "CGAN with pretrained U-net weights on dataset 2",
             "CGAN trained from scratch on dataset 2"]
for (train, test, plot_title) in zip(train_loss, test_loss, ex_titles):
  plot_loss(train, test, plot_title)

In [ ]:
#plot ex2 and ex5
epoch_num = np.arange(len(train_loss[1]))
epoch_num2 = np.arange(len(train_loss[4]))
plt.plot(epoch_num, train_loss[1], label="U-net train on dataset 1")
plt.plot(epoch_num, test_loss[1], label='U-net test on dataset 1')
plt.plot(epoch_num2, train_loss[4], label='U-net train on dataset 2')

plt.xlabel('Num of Epochs')
plt.ylabel("Mean Squared Error")
plt.title("U-net Model Performance")
plt.legend()
plt.show()

In [ ]:
#plot ex3 and ex6
print(len(train_loss[2]))
epoch_num = np.arange(len(train_loss[2]))
epoch_num2 = np.arange(len(train_loss[5]))
plt.plot(epoch_num, train_loss[2], label="CGAN train on dataset 1")
plt.plot(epoch_num, test_loss[2], label='CGAN test on dataset 1')
plt.plot(epoch_num2, train_loss[5], label='CGAN train on dataset 2, U-net pretrained')
# plt.plot(epoch_num2, train_loss[6], label='CGAN train on dataset 2, U-net from scratch')

plt.xlabel('Num of Epoch')
plt.ylabel("Total Loss of CGAN")
plt.title("CGAN Model Performance")
plt.legend()
plt.show()